In [100]:
import networkx as nx
import random
import itertools
import time

In [101]:
def load_gcol(file_path):
    G = nx.Graph() 

    with open(file_path, "r") as file:
        for line in file:
            parts = line.split()
            if parts[0] == 'e':
                node1 = int(parts[1]) 
                node2 = int(parts[2])  
                G.add_edge(node1, node2) 
    return G

In [102]:
def load_graph(file_path):
    G = nx.Graph() 

    with open(file_path, "r") as file:
        for line in file:
            parts = line.split()
            
            node1 = int(parts[0]) 
            node2 = int(parts[1]) 
            G.add_edge(node1, node2) 
    return G

In [103]:
def is_resolving_set(B, G):
    distances = {}
    for v in G.nodes:
        distances[v] = [nx.shortest_path_length(G, v, u) for u in B]
    
    for v1, v2 in itertools.combinations(G.nodes, 2):
        if distances[v1] == distances[v2]:
            return False
    return True

In [104]:
def generate_solution(G):
    B = set()
    while not is_resolving_set(B, G):
        v = random.choice(list(G.nodes))
        B.add(v)
    return B

In [105]:
def Nk(B, k, G):
    V = set(G.nodes)
    if k <= len(B):
        # Izbor k elemenata iz komplementa skupa B
        complement = V - B
        new_sets = []
        for comb in itertools.combinations(complement, k):
            new_set = B - set([random.choice(list(B))])  # Brisanje jednog elementa iz B
            new_set.update(comb)  # Dodavanje novih elemenata iz komplementa
            new_sets.append(new_set)
        return new_sets
    return []

In [106]:
def DeleteLast(B):
    B0 = B.copy()  
    last_element = list(B0)[-1]
    B0.remove(last_element) 
    return B0  

In [107]:
def Shaking(B, k, G):
    neighbors = Nk(B, k, G)
    return random.choice(neighbors) if neighbors else B

In [108]:
def ObjF(B, G):
    distances = {v: [nx.shortest_path_length(G, v, u) for u in B] for v in G.nodes}
    count = 0
    for v1, v2 in itertools.combinations(G.nodes, 2):
        if distances[v1] == distances[v2]:
            count += 1
    return count

In [109]:
def Compare(B0, B00, pmove,G):
    if  len(B00) < len(B0) or ObjF(B00,G) < ObjF(B0,G):
        return True
    elif len(B00) == len(B0) and ObjF(B00,G) > ObjF(B0,G):
        return False
    elif len(B00) == len(B0) and ObjF(B00,G) == ObjF(B0,G) and random.random() < pmove:
        return True
    return False

In [110]:
def LexSort(G, B):
    distances = {v: tuple(nx.shortest_path_length(G, v, u) for u in B) for v in G.nodes}
    sorted_vertices = sorted(G.nodes, key=lambda v: distances[v])
    return sorted_vertices, distances

In [111]:
def IdentifyBlocks(sorted_vertices, distances):
    # Identifikuje blokove (grupe sa istim metričkim koordinatama)
    blocks = []
    current_block = [sorted_vertices[0]]

    for v in sorted_vertices[1:]:
        if distances[v] == distances[current_block[0]]:
            current_block.append(v)
        else:
            if len(current_block) > 1:
                blocks.append(current_block)
            current_block = [v]

    if len(current_block) > 1:
        blocks.append(current_block)

    return blocks

In [112]:
def LocalSearch(B, B00, G):
    improved = True

    while improved:
        improved = False
        objval = ObjF(B00, G)

        for vr in B00:  
           
            z = {v: 0 for v in set(G.nodes) - B00}
             
            B00_minus_vr = B00 - {vr}

            sorted_vertices, distances = LexSort(G, B00_minus_vr)
            blocks = IdentifyBlocks(sorted_vertices, distances)           

            # Ažuriranje z[v] na osnovu blokova (korak 8-12)
            for block in blocks:
                for p in block:
                    for q in block:
                        if(q > p):
                            for v in set(G.nodes) - B00:
                                if nx.shortest_path_length(G, p, v) == nx.shortest_path_length(G, q, v):
                                    z[v] += 1

            # print(z)
            # Pronalazak minimalnog z[v] (korak 13)
            vmin = min(z, key=z.get)

            # Korak 14-18: Ako nađemo poboljšanje DOBRO
            if z[vmin] == 0:
                B = (B00 | {vmin}) - {vr}
                B00 = DeleteLast(B)
                objval = ObjF(B00, G)
                improved = True

            # Korak 19-23: Ako je z[vmin] bolje od trenutnog objval DOBRO
            elif z[vmin] < objval:
                B00 = (B00 | {vmin}) - {vr}
                objval = z[vmin]
                improved = True
    return B

In [113]:
def VNS(G,kmin, kmax, itermax, pmove):
    B = generate_solution(G)
    B0 = DeleteLast(B)
    iter = 0

    k = kmin
    print('Pocetna ',B,B0)
    while itermax > iter:
        iter +=1
              
        B00 = Shaking(B0, k, G)
        B00 = LocalSearch(B, B00, G)
        
        if Compare(B0, B00, pmove, G):
            B0 = B00  
        else:
            if k < kmax:
                k += 1
            else:
                k = kmin
        
        print(f"Iteracija {iter}, najbolji skup {B0}")
    return B0

In [114]:
G = load_graph('grafovi\csp\csp50.txt')
if not nx.is_connected(G):
    print("Graf nije povezan!")
    largest_cc = max(nx.connected_components(G), key=len)
    G = G.subgraph(largest_cc).copy()

Graf nije povezan!


<>:1: SyntaxWarning: invalid escape sequence '\c'
<>:1: SyntaxWarning: invalid escape sequence '\c'
C:\Users\Korisnik\AppData\Local\Temp\ipykernel_465304\513673658.py:1: SyntaxWarning: invalid escape sequence '\c'
  G = load_graph('grafovi\csp\csp50.txt')


In [115]:
param_grid = {
    'kmin':[1,2],
    'kmax': [4],
    'pmove': [0.2],
    'itermax': [3,5]
}

best_score = float('inf')
best_params = None

for combo in itertools.product(*param_grid.values()):
    params = dict(zip(param_grid.keys(), combo))
    
    print(f"Testing: {params}")
    
    start_time = time.time()  # ⏱️ početak
    os = VNS(G,**params)
    score = len(os)
    end_time = time.time()    # ⏱️ kraj
    
    duration = end_time - start_time
    print(f"🔁 Rezultat: {score} | ⏱️ Vrijeme: {duration:.2f} sekundi")
    
    if (score < best_score) or (score == best_score and duration < duration1):
        if(is_resolving_set(os,G)):
            best_score = score
            best_params = params
            duration1 = duration

print("\n✅ Najbolji rezultat:", best_score)
print("📌 Najbolji parametri:", best_params)
print("📌 Vrijeme:", duration1)

Testing: {'kmin': 1, 'kmax': 4, 'pmove': 0.2, 'itermax': 3}
Pocetna  {1, 4, 5, 9, 10, 11, 13, 14, 15, 144, 18, 20, 22, 23, 24, 25, 29, 31, 32, 33, 36, 38, 40, 42, 43, 44, 46, 49, 50, 364} {1, 4, 5, 9, 10, 11, 13, 14, 15, 144, 18, 20, 22, 23, 24, 25, 29, 31, 32, 33, 36, 38, 40, 42, 43, 44, 364, 46, 49}
Iteracija 1, najbolji skup {33, 2, 34, 6, 42, 44, 364, 15, 48}
Iteracija 2, najbolji skup {33, 2, 34, 6, 42, 44, 364, 15, 48}
Iteracija 3, najbolji skup {33, 3, 8, 42, 44, 364, 12, 18}
🔁 Rezultat: 8 | ⏱️ Vrijeme: 0.47 sekundi
Testing: {'kmin': 1, 'kmax': 4, 'pmove': 0.2, 'itermax': 5}
Pocetna  {3, 36, 7, 41, 42, 9, 44, 47, 15, 49, 144, 50, 20, 21, 22, 23, 26, 28} {3, 7, 9, 15, 144, 20, 21, 22, 23, 26, 28, 36, 41, 42, 44, 47, 49}
Iteracija 1, najbolji skup {1, 34, 3, 37, 6, 5, 7, 41, 42, 8, 48}
Iteracija 2, najbolji skup {4, 5, 37, 7, 8, 42, 9, 48, 21}
Iteracija 3, najbolji skup {5, 37, 8, 9, 42, 48, 21, 27}
Iteracija 4, najbolji skup {5, 37, 8, 9, 42, 48, 21, 27}
Iteracija 5, najbolji sku

In [116]:
is_resolving_set([99, 5, 12, 13, 206, 47, 113, 19, 149, 86, 88, 217, 26, 189],G)

NodeNotFound: Target 99 is not in G